# lasagne
* lasagne - это библиотека для написания нейронок произвольной формы на theano
* библиотека низкоуровневая, границы между theano и lasagne практически нет

В качестве демо-задачи выберем то же распознавание чисел, но на большем масштабе задачи
* картинки 28x28
* 10 цифр

In [16]:
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [17]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import *

input_X = T.tensor4("X")
target_y = T.vector("target Y integer",dtype='int32')

Так задаётся архитектура нейронки

### дальше вы могли бы просто
* задать функцию ошибки вручную
* посчитать градиент ошибки по all_weights
* написать updates
* но это долго, а простой шаг по градиенту - не самый лучший смособ оптимизировать веса

Вместо этого, опять используем lasagne

### Вот и всё, пошли её учить
* данных теперь много, поэтому лучше учиться стохастическим градиентным спуском
* для этого напишем функцию, которая бьёт выпорку на мини-батчи (в обычном питоне, не в theano)

In [18]:
# вспомогательная функция, которая возвращает список мини-батчей для обучения нейронки

#на вход
# inputs - тензор из картинок размером (много, 1, 28, 28), например - X_train
# targets - вектор из чиселок - ответов для каждой картинки из X; например - Y_train
# batchsize - одно число - желаемый размер группы

#что нужно сделать
# 1) перемешать данные
# - важно перемешать targets и targets одним и тем же образом, чтобы сохранить соответствие картинки ответу на неё
# 3) побить данные на подгруппы так, чтобы в каждой подгруппе было batchsize картинок и ответов
# - если число картинок не делится на batchsize, одну подгруппу можно вернуть другого размера
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

# Процесс обучения

# Нейронка твоей мечты


* Задача - сделать нейронку, которая получит точность 99% на валидации (validation accuracy)
 * Вариант "is fine too" - 97.5%. 
 * Чем выше, тем лучше.
 
__ В конце есть мини-отчётик, который имеет смысл прочитать вначале и заполнять по ходу работы. __
 

## Что можно улучшить:



 * размер сети
   * бОльше нейронов, 
   * бОльше слоёв, 
   
 * регуляризация - чтобы не переобучалось
   * приплюсовать к функции ошибки какую-нибудь сумму квадратов весов
   * можно сделать вручную, а можно - http://lasagne.readthedocs.org/en/latest/modules/regularization.html
   * Dropout - для борьбы с переобучением
       * `lasagne.layers.DropoutLayer(предыдущий_слой, p=вероятность_занулить)`
   
 * Метод оптимизации - adam, rmsprop, nesterov_momentum, adadelta, adagrad и т.п.
   * сходятся быстрее и иногда - к лучшему оптимуму
   * имеет смысл поиграть с размером батча, количеством эпох и скоростью обучения
   
 * Наконец, можно поиграть с нелинейностями в скрытых слоях
   * tanh, relu, leaky relu, etc

In [44]:
from mnist import load_dataset
X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()

print X_train.shape,y_train.shape,X_val.shape,y_val.shape,X_test.shape,y_test.shape

(50000, 1, 28, 28) (50000,) (10000, 1, 28, 28) (10000,) (10000, 1, 28, 28) (10000,)


In [45]:
import lasagne

input_X = T.tensor4("X")

#размерность входа (None означает "может изменяться")
input_shape = (None,1,28,28)

target_y = T.vector("target Y integer",dtype='int32')

In [46]:
from lasagne.layers import Conv2DLayer, DropoutLayer, DenseLayer, MaxPool2DLayer
from lasagne.init import GlorotUniform


#входной слой (вспомогательный)
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

#<моя архитектура>  


network = input_layer

network = Conv2DLayer(network, num_filters=40, filter_size=(5, 5), nonlinearity=sigmoid, W=GlorotUniform())

network = MaxPool2DLayer(network, pool_size=(2, 2))

network = Conv2DLayer(network, num_filters=40, filter_size=(3, 3), nonlinearity=sigmoid)
    
network = MaxPool2DLayer(network, pool_size=(2, 2))

network = DropoutLayer(network, p=.2)

network = DenseLayer(network, num_units=256, nonlinearity=sigmoid)

network = DropoutLayer(network, p=.5)
   
network = DenseLayer(network, num_units=10, nonlinearity=softmax)    

dense_output = network


#ВЫХОДНОЙ полносвязный слой, который принимает на вход dense_1 и имеет 10 нейронов -по нейрону на цифру
#нелинейность - softmax - чтобы вероятности всех цифр давали в сумме 1
#dense_output = lasagne.layers.DenseLayer(dense_9,num_units = 10,
#                                       nonlinearity = lasagne.nonlinearities.softmax,
#                                       name='output')

In [47]:
lasagne.layers.get_all_layers(dense_output)

In [48]:
lasagne.layers.count_params(dense_output)

274306

In [49]:
#предсказание нейронки (theano-преобразование)
y_predicted = lasagne.layers.get_output(dense_output)

In [50]:
#все веса нейронки (shared-переменные)
all_weights = lasagne.layers.get_all_params(dense_output)
print all_weights

[W, b, W, b, W, b, W, b]


In [51]:
from lasagne.regularization import regularize_layer_params_weighted, l2, l1
from lasagne.regularization import regularize_layer_params

In [52]:
#функция ошибки - средняя кроссэнтропия
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()

#<возможно добавить регуляризатор>

#layers = {dense_1: 0.1, dense_2: 0.1, dense_3: 0.1, dense_4: 0.1}
#penalty = lasagne.regularization.regularize_layer_params_weighted(layers, l1) 
#penalty = regularize_layer_params(dense_1, l2) * 1e-4  
    
#loss = loss + penalty





accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#сразу посчитать словарь обновлённых значений с шагом по градиенту, как раньше
#updates_sgd = <поиграться с методами>

#updates_sgd = lasagne.updates.rmsprop(loss, all_weights,learning_rate=0.01)
#updates_sgd = lasagne.updates.sgd(loss, all_weights,learning_rate=0.01)
updates_sgd = lasagne.updates.adamax(loss, all_weights,learning_rate=0.01)


In [53]:
#функция, которая обучает сеть на 1 шаг и возвращащет значение функции потерь и точности
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#функция, которая считает точность
accuracy_fun = theano.function([input_X,target_y],accuracy)

In [54]:
%%time

#итерации обучения

num_epochs = 30  #10 сколько_эпох #количество проходов по данным

batch_size =  25  #50 сколько_картинок_в_минибатче #размер мини-батча

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Epoch 1 of 30 took 124.418s
  training loss (in-iteration):		2.127599
  train accuracy:		20.26 %
  validation accuracy:		84.02 %
Epoch 2 of 30 took 118.507s
  training loss (in-iteration):		0.232760
  train accuracy:		92.96 %
  validation accuracy:		96.36 %
Epoch 3 of 30 took 109.601s
  training loss (in-iteration):		0.114289
  train accuracy:		96.48 %
  validation accuracy:		97.31 %
Epoch 4 of 30 took 108.978s
  training loss (in-iteration):		0.086378
  train accuracy:		97.37 %
  validation accuracy:		97.85 %
Epoch 5 of 30 took 108.331s
  training loss (in-iteration):		0.073002
  train accuracy:		97.78 %
  validation accuracy:		97.71 %
Epoch 6 of 30 took 109.332s
  training loss (in-iteration):		0.062927
  train accuracy:		98.01 %
  validation accuracy:		98.36 %
Epoch 7 of 30 took 107.462s
  training loss (in-iteration):		0.055411
  train accuracy:		98.25 %
  validation accuracy:		98.31 %
Epoch 8 of 30 took 107.873s
  training loss (in-iteration):		0.051296
  train accuracy:		98.40 %


In [55]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

Final results:
  test accuracy:		98.75 %
